In [1]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm
import config as cfg
import load_s3dis_preorder as ds
import resnet_seg as res
import utils as utl

/home/wangh0j/miniconda3/envs/deepgcn2/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wangh0j/miniconda3/envs/deepgcn2/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wangh0j/miniconda3/envs/deepgcn2/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wangh0j

In [2]:
import pyvista as pv
import numpy as np
from hilbertcurve.hilbertcurve import HilbertCurve

In [3]:
def batched_index_select(inputs, index):
    """

    :param inputs: torch.Size([batch_size, num_dims, num_vertices, 1])
    :param index: torch.Size([batch_size, num_vertices, k])
    :return: torch.Size([batch_size, num_dims, num_vertices, k])
    """
    batch_size, num_dims, num_vertices, _ = inputs.shape
    k = index.shape[2]
    idx = torch.arange(0, batch_size) * num_vertices
    idx = idx.view(batch_size, -1)

    inputs = inputs.transpose(2, 1).contiguous().view(-1, num_dims)
    index = index.contiguous().view(batch_size, -1) + idx.type(index.dtype).to(inputs.device)
    index = index.view(-1)

    return torch.index_select(inputs, 0, index).contiguous().view(batch_size, -1, num_dims).transpose(2, 1).contiguous().view(batch_size, num_dims, -1, k)

In [4]:
pos = torch.randn(4, 9, n_points)
print(pos)

batch, n_dim, n_points = pos.shape

NameError: name 'n_points' is not defined

In [ ]:
root_dir = '/data/sfc/S3DIS'
batch_size = 32
in_channels = 9
kernel_size = 15

In [ ]:
phases = ['train', 'test']
datasets, dataloaders, num_classes = ds.get_s3dis_dataloaders(root_dir=root_dir,
                                                              phases=phases,
                                                              batch_size=batch_size,
                                                              category=5)

In [ ]:
device = torch.device('cuda:0')
model = res.resnet18(in_channels=in_channels, num_classes=num_classes,
                         kernel_size=kernel_size).to(device)

In [ ]:
dataloader_iter = iter(dataloaders['train'])
data = next(dataloader_iter)

In [ ]:
pos = data.pos
print('pos shape is ', pos.shape)
# dense points
points = torch.cat(((pos+1)/2, data.x[:, :, :3]), dim=2)[0]
points = points.numpy()
print('points dtype', points.shape, points.dtype)

In [ ]:
## for pre process data


## code below is for visualzaiton SFC curve

In [ ]:
import pyvista as pv
plotter = pv.Plotter(shape=(2, 2))
plotter.link_views() 
for i in range(4):
    points_voxelized = multi_pos[ :, :].numpy()
    point_cloud = pv.PolyData(points_voxelized)
    plotter.subplot(i//2, i%2)
    plotter.show_grid()
    plotter.add_mesh(point_cloud,point_size=10., render_points_as_spheres=True)
    cells = np.full((points_voxelized.shape[0] - 1, 3), 2, dtype=np.int)
    cells[:, 1] = np.arange(0, points_voxelized.shape[0] - 1, dtype=np.int)
    cells[:, 2] = np.arange(1, points_voxelized.shape[0], dtype=np.int)
    point_cloud.lines = cells
    plotter.add_mesh(point_cloud, point_size=10., 
                     line_width=2., render_points_as_spheres=True)

plotter.show()

In [ ]:
import pyvista as pv
plotter = pv.Plotter(shape=(1, 2))
point_cloud = pv.PolyData(point[:, :3])
plotter.subplot(0, 0)
plotter.show_grid()
plotter.add_mesh(point_cloud,point_size=10., render_points_as_spheres=True)

point_cloud = pv.PolyData(points_voxel[:, :3])
plotter.subplot(0, 1)
plotter.add_mesh(point_cloud,point_size=10., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()

In [ ]:

voxel_size = 0.02
points = points_voxel
points_quantized = np.floor(points[:, :3] / voxel_size).astype(int)
points_quantized, quantization_indices = np.unique(points_quantized, return_index=True, axis=0)
points_voxelized = points[quantization_indices, :]
points_voxelized.shape
plotter = pv.BackgroundPlotter()
plotter.enable_eye_dome_lighting()
# dense points
point_cloud = pv.PolyData(point[:, :3])
p.subplot(0, 0)
plotter.add_mesh(point_cloud,point_size=10., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()


point_cloud = pv.PolyData(point_voxel[:, :3])
p.subplot(0, 0)
plotter.add_mesh(point_cloud,point_size=10., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()

# voxelized points
point_cloud = pv.PolyData(points_voxelized[:, :3])
plotter.add_mesh(point_cloud, point_size=30., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()

In [ ]:
point_cloud = pv.PolyData(points[:, :3])
point_cloud['color'] = points[:, 3:6]
point_cloud

In [ ]:
voxel_size = 0.02
points_quantized = np.floor(points[:, :3] / voxel_size).astype(int)
points_quantized, quantization_indices = np.unique(points_quantized, return_index=True, axis=0)
points_voxelized = points[quantization_indices, :]
points_voxelized.shape
plotter = pv.BackgroundPlotter()
plotter.enable_eye_dome_lighting()
# dense points
point_cloud = pv.PolyData(points[:, :3])
point_cloud['color'] = points[:, 3:6]
plotter.add_mesh(point_cloud, scalars='color', rgb=True, point_size=10., render_points_as_spheres=True)
# voxelized points
point_cloud = pv.PolyData(points_voxelized[:, :3])
point_cloud['color'] = points_voxelized[:, 3:6]
plotter.add_mesh(point_cloud, scalars='color', rgb=True, point_size=30., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()

In [ ]:
p = np.ceil(np.log2(points_quantized.max())).astype(int)
print('p is ', p)
hilbert_curve = HilbertCurve(p, 3)
hilbert_dist = np.zeros(points_quantized.shape[0])
for i in range(points_quantized.shape[0]):
    hilbert_dist[i] = hilbert_curve.distance_from_coordinates(points_quantized[i, :])
hilbert_idx = np.argsort(hilbert_dist)
points_voxelized = points_voxelized[hilbert_idx, :]
points_quantized = points_quantized[hilbert_idx, :]
plotter = pv.BackgroundPlotter()
plotter.enable_eye_dome_lighting()
# voxelized points
point_cloud = pv.PolyData(points_voxelized[:, :3])
point_cloud['color'] = points_voxelized[:, 3:6]
cells = np.full((points_voxelized.shape[0] - 1, 3), 2, dtype=np.int)
cells[:, 1] = np.arange(0, points_voxelized.shape[0] - 1, dtype=np.int)
cells[:, 2] = np.arange(1, points_voxelized.shape[0], dtype=np.int)
point_cloud.lines = cells
plotter.add_mesh(point_cloud, scalars='color', rgb=True, point_size=30., 
                 line_width=5., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()


In [ ]:
def inside_cube(point, mins, maxs):
    isin = True
    for i in range(len(point)):
        if point[i] < mins[i] or point[i] > maxs[i]:
            isin = False
    return isin
kernel_points = np.mgrid[0:3, 0:3, 0:3].reshape((3, -1)).T
hilbert_dist = np.zeros(kernel_points.shape[0])
for i in range(kernel_points.shape[0]):
    hilbert_dist[i] = hilbert_curve.distance_from_coordinates(kernel_points[i, :])
idx = np.argsort(hilbert_dist)
kernel_points = kernel_points[idx, :]
center_point = points_quantized[100000, :]
print(center_point)
kernel_local = kernel_points + center_point - 1
kernel_min = kernel_local.min(axis=0)
kernel_max = kernel_local.max(axis=0)
points_in = np.array([points_quantized[i, :] for i in range(points_quantized.shape[0]) 
                      if inside_cube(points_quantized[i, :], kernel_min, kernel_max)])
points_in
plotter = pv.BackgroundPlotter()
plotter.enable_eye_dome_lighting()
point_cloud = pv.PolyData(points_quantized)
# point_cloud['color'] = np.arange(points_quantized.shape[0])
# cells = np.full((points_quantized.shape[0] - 1, 3), 2, dtype=np.int)
# cells[:, 1] = np.arange(0, points_quantized.shape[0] - 1, dtype=np.int)
# cells[:, 2] = np.arange(1, points_quantized.shape[0], dtype=np.int)
# point_cloud.lines = cells
plotter.add_mesh(point_cloud, color='maroon', point_size=10., line_width=5, render_points_as_spheres=True)
# plotter.add_mesh(point_cloud, scalar='color', point_size=10., line_width=5, render_points_as_spheres=True)
point_cloud = pv.PolyData(center_point)
plotter.add_mesh(point_cloud, color='green', point_size=30., render_points_as_spheres=True)
point_cloud = pv.PolyData(points_in)
cells = np.full((points_in.shape[0] - 1, 3), 2, dtype=np.int)
cells[:, 1] = np.arange(0, points_in.shape[0] - 1, dtype=np.int)
cells[:, 2] = np.arange(1, points_in.shape[0], dtype=np.int)
point_cloud.lines = cells
plotter.add_mesh(point_cloud, color='green', point_size=20., line_width=5.0, render_points_as_spheres=True)
box = pv.Box(bounds=(kernel_min[0], kernel_max[0], kernel_min[1], kernel_max[1], kernel_min[2], kernel_max[2]))
plotter.add_mesh(box, color='blue', show_edges=True, opacity=0.85)
plotter.show_grid()
plotter.show()